In [2]:
from langchain.chains.summarize import load_summarize_chain
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import PyPDFLoader

In [4]:
loader = PyPDFLoader("10Q.pdf")
pages = loader.load()

In [6]:
text = ''
for page in pages:
    text+=page.page_content

In [23]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=5000, chunk_overlap=20)
chunks = text_splitter.create_documents([text])

In [24]:
len(chunks)

14

In [9]:
from langchain.chat_models import ChatOpenAI
import os
os.environ["OPENAI_API_KEY"] = "API_KEY"


In [11]:
llm=ChatOpenAI(model_name='gpt-3.5-turbo')

In [13]:
llm.get_num_tokens(text)

18141

In [19]:
from llamaapi import LlamaAPI

# Replace 'Your_API_Token' with your actual API token
llama = LlamaAPI("llm_api")

In [21]:
from langchain_experimental.llms import ChatLlamaAPI
llm =ChatLlamaAPI(client=llama,)

In [32]:
from langchain import HuggingFaceHub

llm = HuggingFaceHub(
    repo_id = "amitesh11/bart-finance-pegasus",
    huggingfacehub_api_token = "hug-api"
)

C:\Users\kinga\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
from langchain.llms import Llama
api_key = 'llama_api'
model_version = 'llama2-70b'
llm = Llama(
    api_key=api_key,
    model_version=model_version
)

In [ ]:
chain = load_summarize_chain(
    llm,
    chain_type='map_reduce',
    verbose = False
)

summary = chain.run(chunks)

In [ ]:
summary

In [ ]:
from langchain import PromptTemplate
chunks_prompt = """
You are a financial expert and summarize the following infomation into a meaningful few lines:
information: `{text}`
summary:
"""
map_prompt_temp = PromptTemplate(input_variables=['text'],template=chunks_prompt)


final_combine_prompt ="""
Summarize all the information as a expert financial advisor in the following format:

1. Business overview:  Include information such as the company's formation/incorporation date, headquarters location, business description, employee count, latest revenues, stock exchange listing and market capitalization, number of offices and location, and details on their clients/customers.
2. Business Segment Overview:
. Extract the revenue percenteage of each component ( verticals, products, segments and sections) as a part of the total revenue.
. Performance: Evaluate the performance of each component by comparing the current year's sales/revenue and market share with the previous year's numvers.
. Sales Increase/Descrease Explanation: Explain the causes of the icreaase or descrease in the performacne of each component.
3. Breakdown of sales and revneue by geography, specifying the percentage contribution of each region of the total sales.
4. Summarize geographical data, such as workforce, clients, and offices, and outline the company's regional plans for exapnasion or reduction.
5. Analuze and expkain regional sales fluctutations, icluding a geographical sales breakdown to indentify sales trends..
6. Year over year sales incerase or decline and reasons for the change.
7. Summary of rationale and considerations ( risks and mitigation factors)
8.SWOT Analysis
9. Information about credit rating/credit rating change/change in the rating outlook.

information: `{text}`
"""
final_combine_prompt_template=PromptTemplate(input_variables=['text'],
                                             template=final_combine_prompt)

In [ ]:
summary_chain = load_summarize_chain(
    llm=llm,
    chain_type='map_reduce',
    map_prompt=map_prompt_temp,
    combine_prompt=final_combine_prompt_template,
    verbose=False
)
output = summary_chain.run(chunks)

In [ ]:
output